In [1]:
import pandas as pd
import openpyxl
import os
from copy import copy
from openpyxl.styles import PatternFill, Font, Alignment, Border
import re
import subprocess
import sys

column_mapping = {
    'Emp id': 'EID',
    'Hire Date (DD-MMM-YYYY)': 'DOJ',
    'FULL_NAME': 'EMPLOYEE FULL NAME_EN',
    'Họ tên': 'EMPLOYEE FULL NAME_VN',
    'Gender': 'GENDER',
    'Giới tính': 'GENDER',
    'DOB (DD-MMM-YYYY)': 'DOB',
    'Dân tộc': 'ETHNIC',
    'Nơi sinh': 'BIRTH PLACE',
    'Số CMND/CCCD:': 'ID CARD',
    'Ngày cấp CMND/CCCD': 'ISSUED DATE',
    'Nơi cấp CCCD/CMND': 'ISSUED PLACE',
    'Số CMND Cũ (Nếu Có)': 'OLD ID CARD NO.',
    'Địa chỉ thường trú:': 'PERMANENT ADDRESS',
    'Địa chỉ tạm trú': 'TEMPORARY ADDRESS',
    'Highest Education': 'EDUCATION',
    'Tình trạng hôn nhân': 'MARITAL STATUS',
    'Phone Number': 'MOBILE PHONE NO',
    'Email - Work': 'CNX EMAIL',
    'Home E-mail address': 'PERSONAL EMAIL',
    'Số điện thoại người liên hệ trong trường hợp khẩn cấp': 'EMERGENCY CONTACT',
    'Số Bảo Hiểm Xã Hội': 'SOCIAL INSURANCE NUMBER',
    'Mã số thuế': 'PERSONAL TAX CODE',
    'Số tài khoản ngân hàng nhận lương': 'BANK ACCOUNT',
    'Tên ngân hàng': 'BANK NAME',
    'Base Pay': 'BASE SALARY',
    'Complexity Allowance': 'COMPLEXCITY ALLOWANCE',
    'Position Allowance': 'POSITION ALLOWANCE',
    'Meal Allowance - Monthly': 'MEAL ALLOWANCE',
    'Business Title': 'BUSINESS_TITLE_EN',
    'Vị trí': 'BUSINESS_TITLE_VN',
    'SUPERVISOR_FULL_NAME': 'SUPERVISOR',
    'Career Level': 'CAREER LEVEL',
    'Họ và tên chủ hộ': 'Họ tên chủ hộ',
    'Giới tính chủ hộ': 'Giới tính chủ hộ',
    'Ngày sinh của chủ hộ': 'DOB chủ hộ',
    'Mối quan hệ giữa chủ hộ và bạn': 'Mối quan hệ giữa NLĐ với chủ hộ',
    "MSA Client": "MSA Client",
    "Contract ID": "PROBATION CONTRACT NO",
    "Process": "PROJECT",
    'ORIGINAL_DATE_OF_HIRE': 'DOJ',
    "Start Date (DD-MMM-YYYY)" : "FROM",
    "End Date (DD-MMM-YYYY)" : "TO",
    "Bệnh viện muốn đăng ký BHYT": "HOSPITAL NAME ",
    "Legislation Code / Country": "NATIONALITY"
}

template_file_path = os.path.abspath("workday_data.xlsx")
merge_hr_files = "merge_hr_files.py"

master_data_sheet_name="Masterdata_PSteam"
employee_master_sheet_name="Employee Master"
hiring_data_sheet_name="Hiring_data"
lineup_pass_import_sheet_name="LINEUP_PASS_IMPORT"
termination_sheet_name="Termination"
permission_sheet_name="Permission"

master_data_skip_rows = 11
employee_master_skip_rows=11
hiring_data_skip_rows=1
lineup_pass_import_skip_rows=0
termination_skip_rows=10
permission_skip_rows=0

master_data = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=master_data_sheet_name, skiprows=master_data_skip_rows)
employee_master = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=employee_master_sheet_name, skiprows=employee_master_skip_rows)
hiring_data = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=hiring_data_sheet_name,  skiprows=hiring_data_skip_rows)
lineup_pass_import = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=lineup_pass_import_sheet_name,  skiprows=lineup_pass_import_skip_rows)
termination = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                            sheet_name=termination_sheet_name, skiprows=termination_skip_rows)
permission = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                            sheet_name=permission_sheet_name, skiprows=permission_skip_rows)

merged_data_first = pd.merge(
    hiring_data,
    employee_master,
    left_on='Emp id',
    right_on='EMPLOYEE_NUMBER',
    how='left'
)
columns_to_drop = [col for col in merged_data_first.columns if col.endswith('_y')]
merged_data_first = merged_data_first.drop(columns=columns_to_drop)
# Đổi tên các cột '_x' thành tên gốc
merged_data_first.columns = [col.replace('_x', '') for col in merged_data_first.columns]
# merged_data_first.to_excel("data_first.xlsx", index=False)
merged_data_second = pd.merge(
    merged_data_first,
    lineup_pass_import,
    left_on='National ID (SSN/SIN) (National Identifiers)',
    right_on='Số CMND/CCCD:',
    how='left'
)
columns_to_drop = [col for col in merged_data_second.columns if col.endswith('_y')]
merged_data_second = merged_data_second.drop(columns=columns_to_drop)
# Đổi tên các cột '_x' thành tên gốc
merged_data_second.columns = [col.replace('_x', '') for col in merged_data_second.columns]
# merged_data_second.to_excel("data_second.xlsx", index=False)
# merged_data_second = pd.merge(
#     merged_data_second,
#     termination,
#     left_on='Emp id',
#     right_on='EMPLOYEE_ID',
#     how='left'
# )

# columns = merged_data_second.columns
# columns_to_drop = [col for col in columns if col.endswith('_y')]
# merged_data_second = merged_data_second.drop(columns=columns_to_drop)
# merged_data_second = merged_data_second.rename(columns=lambda x: x.replace('_x', '') if x.endswith('_x') else x)
try:
    master_data_columns = [col for col in master_data.columns if not str(col).startswith('Unnamed')]
    merged_data_second = merged_data_second.dropna(how='all')
    master_data = master_data.dropna().loc[:, ~master_data.columns.str.contains('^Unnamed')]
    mapped_columns = {}
    for merged_col, master_col in column_mapping.items():
        if merged_col in merged_data_second.columns:
            mapped_columns[master_col] = merged_data_second[merged_col]
        # else:
        #     print("Line 91: Column '{}' not found in merged data".format(merged_col))

    # Tạo DataFrame mới từ dictionary bằng pd.concat
    mapped_df = pd.concat(mapped_columns, axis=1)

    # Thêm các cột còn thiếu trong mapped_df và điền giá trị NaN
    for col in master_data_columns:
        if col not in mapped_df.columns:
            # print("Line 99: Column '{}' not found in merged data".format(col))
            mapped_df[col] = pd.NA

    # Sắp xếp các cột của mapped_df theo thứ tự cột của master_data
    mapped_df = mapped_df[master_data_columns]

    # Kết hợp dữ liệu từ mapped_df vào master_df
    master_df = pd.concat([master_data, mapped_df], ignore_index=True)
    output_file = 'master_data.xlsx'
    final_df = pd.merge(
        master_df,
        permission,
        on="MSA Client",
        how='left')
    final_df.to_excel(output_file, sheet_name='master_data', index=False)

    # Copy style here

    if 'SSO' not in final_df.columns:
        raise ValueError("Cột 'SSO' không tồn tại trong master_data")

    # Tạo thư mục để lưu các file Excel (nếu chưa có)
    hr_files_dir = "hr_files"
    os.makedirs(hr_files_dir, exist_ok=True)

    # Duyệt theo từng nhóm 'sso'
    for sso, hr_data in final_df.groupby('SSO'):
        # Tạo tên file an toàn
        safe_sso = re.sub(r'[<>:"/\\|?*]', '_', str(sso))
        output_file = os.path.join(hr_files_dir, f"HR_{safe_sso}.xlsx")

        # Xuất dữ liệu ra file Excel
        hr_data.to_excel(output_file, index=False)
        print(f"Đã tạo file: {output_file} với {len(hr_data)} hàng")

    # subprocess.run([sys.executable, merge_hr_files], capture_output=True, text=True)
    # open(os.path.join(logs_path, 'merge_log.txt'), 'a')
    # open(os.path.join(logs_path, 'merge_error.log'), 'a')
    # open(os.path.join(logs_path, 'watchdog_log.txt'), 'a')
except Exception as e:
    print(e)